In [1]:
import os
os.getcwd()



'C:\\Users\\ELITEBOOK 30 G8\\OneDrive\\Desktop\\Personal\\Data Analysis\\Physical Data Training - PQPLUS\\Portfolio Lab\\Tokenomics_Health Project'

In [2]:
os.listdir()


['.ipynb_checkpoints', 'config', 'data', 'Notebooks', 'Untitled.ipynb']

In [3]:
import pandas as pd
pd.read_csv("config/token_master.csv").head()


,token_name,symbol,coingecko_id,chain,contract_address,tier,category,notes
0,Uniswap,UNI,uniswap,ethereum,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,A,DEX,NaN
1,Chainlink,LINK,chainlink,ethereum,0x514910771af9ca656af840dff83e8264ecf986ca,A,Infrastructure & Middleware,NaN
2,Aave,AAVE,aave,ethereum,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,A,Lending & Borrowing,NaN
3,Maker,MKR,maker,ethereum,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,A,Lending & Borrowing,NaN
4,Lido Dao,LDO,lido-dao,ethereum,0x5a98fcbea516cf06857215779fd812ca3bef1b32,A,Liquid staking,NaN


In [4]:
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timezone

pd.set_option("display.max_columns", 50)


In [5]:
tm = pd.read_csv("config/token_master.csv")
tm.head()


,token_name,symbol,coingecko_id,chain,contract_address,tier,category,notes
0,Uniswap,UNI,uniswap,ethereum,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,A,DEX,NaN
1,Chainlink,LINK,chainlink,ethereum,0x514910771af9ca656af840dff83e8264ecf986ca,A,Infrastructure & Middleware,NaN
2,Aave,AAVE,aave,ethereum,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,A,Lending & Borrowing,NaN
3,Maker,MKR,maker,ethereum,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,A,Lending & Borrowing,NaN
4,Lido Dao,LDO,lido-dao,ethereum,0x5a98fcbea516cf06857215779fd812ca3bef1b32,A,Liquid staking,NaN


In [6]:
missing_ids = tm[tm["coingecko_id"].isna() | (tm["coingecko_id"].astype(str).str.strip() == "")]
missing_ids[["token_name", "symbol", "coingecko_id"]]


,token_name,symbol,coingecko_id


In [7]:
COINGECKO_BASE = "https://api.coingecko.com/api/v3"

def cg_get(endpoint, params=None, sleep_s=1.2):
    """
    Calls CoinGecko API with polite sleeping to reduce rate limit errors.
    """
    if params is None:
        params = {}

    url = f"{COINGECKO_BASE}{endpoint}"
    r = requests.get(url, params=params, timeout=30)

    # Handle rate limit
    if r.status_code == 429:
        time.sleep(10)
        r = requests.get(url, params=params, timeout=30)

    r.raise_for_status()
    time.sleep(sleep_s)
    return r.json()


In [8]:
test_id = tm.loc[0, "coingecko_id"]  # first row token
test_id


'uniswap'

In [9]:
data = cg_get(
    f"/coins/{test_id}",
    params={
        "localization":"false",
        "tickers":"false",
        "market_data":"true",
        "community_data":"false",
        "developer_data":"false",
        "sparkline":"false"
    }
)

# Peek at what keys exist
list(data.keys())


['id',
 'symbol',
 'name',
 'web_slug',
 'asset_platform_id',
 'platforms',
 'detail_platforms',
 'block_time_in_minutes',
 'hashing_algorithm',
 'categories',
 'preview_listing',
 'public_notice',
 'additional_notices',
 'description',
 'links',
 'image',
 'country_origin',
 'genesis_date',
 'contract_address',
 'sentiment_votes_up_percentage',
 'sentiment_votes_down_percentage',
 'watchlist_portfolio_users',
 'market_cap_rank',
 'market_data',
 'status_updates',
 'last_updated']

In [10]:
def extract_snapshot(coingecko_id):
    data = cg_get(
        f"/coins/{coingecko_id}",
        params={
            "localization":"false",
            "tickers":"false",
            "market_data":"true",
            "community_data":"false",
            "developer_data":"false",
            "sparkline":"false"
        }
    )

    md = data.get("market_data", {})
    out = {
        "coingecko_id": coingecko_id,
        "pulled_at_utc": datetime.now(timezone.utc).isoformat(),
        "current_price_usd": md.get("current_price", {}).get("usd", np.nan),
        "market_cap_usd": md.get("market_cap", {}).get("usd", np.nan),
        "volume_24h_usd": md.get("total_volume", {}).get("usd", np.nan),
        "circulating_supply": md.get("circulating_supply", np.nan),
        "max_supply": md.get("max_supply", np.nan),
        "fdv_usd": md.get("fully_diluted_valuation", {}).get("usd", np.nan),
    }
    return out


In [12]:
snapshots = []
errors = []

for _, row in tm.iterrows():
    cid = str(row["coingecko_id"]).strip()
    try:
        snapshots.append(extract_snapshot(cid))
    except Exception as e:
        errors.append({"coingecko_id": cid, "error": str(e)})

market_snapshot = pd.DataFrame(snapshots)

market_snapshot.head(), len(errors)


(      coingecko_id                     pulled_at_utc  current_price_usd  \
 0          uniswap  2026-01-19T12:34:30.906620+00:00           4.980000   
 1        chainlink  2026-01-19T12:34:45.019802+00:00          12.750000   
 2             aave  2026-01-19T12:34:47.605564+00:00         163.460000   
 3         lido-dao  2026-01-19T12:35:14.316675+00:00           0.546995   
 4  curve-dao-token  2026-01-19T12:35:17.069355+00:00           0.390634   
 
    market_cap_usd  volume_24h_usd  circulating_supply    max_supply  \
 0      3160631041       382896158        6.346104e+08  1.000000e+09   
 1      9024026753       648920230        7.081000e+08  1.000000e+09   
 2      2481823459       375268933        1.518357e+07  1.600000e+07   
 3       462970893        56720533        8.465668e+08  1.000000e+09   
 4       573741000        74930533        1.468781e+09  3.030303e+09   
 
        fdv_usd  
 0   4480843112  
 1  12744001030  
 2   2615272322  
 3    546880520  
 4    915416077  ,

In [18]:
pd.DataFrame(errors)


,coingecko_id,error
0,maker,429 Client Error: Too Many Requests for url: h...
1,1inch,429 Client Error: Too Many Requests for url: h...
2,sushi,429 Client Error: Too Many Requests for url: h...
3,ftx-token,429 Client Error: Too Many Requests for url: h...
4,ethos,429 Client Error: Too Many Requests for url: h...


In [14]:
def fetch_market_history_daily(coingecko_id, days=365):
    data = cg_get(
        f"/coins/{coingecko_id}/market_chart",
        params={"vs_currency":"usd", "days":days, "interval":"daily"}
    )

    prices = pd.DataFrame(data["prices"], columns=["ts_ms", "price_usd"])
    mcaps  = pd.DataFrame(data["market_caps"], columns=["ts_ms", "market_cap_usd"])
    vols   = pd.DataFrame(data["total_volumes"], columns=["ts_ms", "volume_usd"])

    df = prices.merge(mcaps, on="ts_ms").merge(vols, on="ts_ms")
    df["date"] = pd.to_datetime(df["ts_ms"], unit="ms", utc=True).dt.date
    df["coingecko_id"] = coingecko_id
    df = df.drop(columns=["ts_ms"])

    # Ensure one row per day
    df = df.groupby(["coingecko_id", "date"], as_index=False).mean(numeric_only=True)
    return df


In [15]:
history_frames = []
history_errors = []

for _, row in tm.iterrows():
    cid = str(row["coingecko_id"]).strip()
    try:
        history_frames.append(fetch_market_history_daily(cid, days=365))
    except Exception as e:
        history_errors.append({"coingecko_id": cid, "error": str(e)})

market_history_daily = pd.concat(history_frames, ignore_index=True)

market_history_daily.head(), len(history_errors)


(  coingecko_id        date  price_usd  market_cap_usd    volume_usd
 0      uniswap  2025-01-20  13.275081    7.997656e+09  6.670297e+08
 1      uniswap  2025-01-21  13.469417    8.100521e+09  7.125942e+08
 2      uniswap  2025-01-22  13.487985    8.099307e+09  4.728167e+08
 3      uniswap  2025-01-23  12.866830    7.729917e+09  3.142100e+08
 4      uniswap  2025-01-24  12.797972    7.688804e+09  3.640611e+08,
 5)

In [22]:
meta_cols = ["coingecko_id","token_name","symbol","tier","category","chain"]

market_snapshot = market_snapshot.merge(tm[meta_cols], on="coingecko_id", how="left")
market_history_daily = market_history_daily.merge(tm[meta_cols], on="coingecko_id", how="left")

market_snapshot.head()


,coingecko_id,pulled_at_utc,current_price_usd,market_cap_usd,volume_24h_usd,circulating_supply,max_supply,fdv_usd,token_name_x,symbol_x,tier_x,category_x,chain_x,token_name_y,symbol_y,tier_y,category_y,chain_y
0,uniswap,2026-01-19T12:34:30.906620+00:00,4.980000,3160631041,382896158,6.346104e+08,1.000000e+09,4480843112,Uniswap,UNI,A,DEX,ethereum,Uniswap,UNI,A,DEX,ethereum
1,chainlink,2026-01-19T12:34:45.019802+00:00,12.750000,9024026753,648920230,7.081000e+08,1.000000e+09,12744001030,Chainlink,LINK,A,Infrastructure & Middleware,ethereum,Chainlink,LINK,A,Infrastructure & Middleware,ethereum
2,aave,2026-01-19T12:34:47.605564+00:00,163.460000,2481823459,375268933,1.518357e+07,1.600000e+07,2615272322,Aave,AAVE,A,Lending & Borrowing,ethereum,Aave,AAVE,A,Lending & Borrowing,ethereum
3,lido-dao,2026-01-19T12:35:14.316675+00:00,0.546995,462970893,56720533,8.465668e+08,1.000000e+09,546880520,Lido Dao,LDO,A,Liquid staking,ethereum,Lido Dao,LDO,A,Liquid staking,ethereum
4,curve-dao-token,2026-01-19T12:35:17.069355+00:00,0.390634,573741000,74930533,1.468781e+09,3.030303e+09,915416077,Curve DAO,CRV,B,DEX,ethereum,Curve DAO,CRV,B,DEX,ethereum


In [23]:
import os
os.makedirs("data/processed", exist_ok=True)

market_snapshot.to_csv("data/processed/market_snapshot.csv", index=False)
market_history_daily.to_csv("data/processed/market_history_daily.csv", index=False)

print("Saved:")
print("data/processed/market_snapshot.csv")
print("data/processed/market_history_daily.csv")


Saved:
data/processed/market_snapshot.csv
data/processed/market_history_daily.csv


In [24]:
import time
import requests

COINGECKO_BASE = "https://api.coingecko.com/api/v3"

def cg_get(endpoint, params=None, base_sleep=1.5, max_retries=5):
    """
    CoinGecko GET with exponential backoff for 429 rate limits.
    - base_sleep: normal pause between calls
    - max_retries: how many times to retry on 429
    """
    if params is None:
        params = {}
    url = f"{COINGECKO_BASE}{endpoint}"

    for attempt in range(max_retries):
        r = requests.get(url, params=params, timeout=30)

        if r.status_code == 429:
            # exponential backoff: 5s, 10s, 20s, 40s...
            wait = 5 * (2 ** attempt)
            print(f"Rate limited (429). Waiting {wait}s then retrying... ({attempt+1}/{max_retries})")
            time.sleep(wait)
            continue

        r.raise_for_status()
        time.sleep(base_sleep)
        return r.json()

    raise Exception(f"Failed after {max_retries} retries due to repeated rate limits: {url}")


In [20]:
failed_ids = [e["coingecko_id"] for e in errors]
failed_ids


['maker', '1inch', 'sushi', 'ftx-token', 'ethos']

In [21]:
recovered = []
still_failed = []

for cid in failed_ids:
    try:
        recovered.append(extract_snapshot(cid))
    except Exception as e:
        still_failed.append({"coingecko_id": cid, "error": str(e)})

pd.DataFrame(recovered).head(), pd.DataFrame(still_failed)


(  coingecko_id                     pulled_at_utc  current_price_usd  \
 0        maker  2026-01-19T13:02:10.700204+00:00        1537.820000   
 1        1inch  2026-01-19T13:02:14.583278+00:00           0.145324   
 2        sushi  2026-01-19T13:02:17.662026+00:00           0.305588   
 3    ftx-token  2026-01-19T13:02:20.620067+00:00           0.496703   
 4        ethos  2026-01-19T13:02:24.111957+00:00           0.000632   
 
    market_cap_usd  volume_24h_usd  circulating_supply    max_supply    fdv_usd  
 0             0.0          168295        0.000000e+00  1.005577e+06  152599623  
 1     203475990.0        22879453        1.400627e+09  1.500000e+09  217912420  
 2      83450070.0        23911143        2.730123e+08           NaN   87932360  
 3             0.0         4174797        0.000000e+00  3.288951e+08  163235993  
 4        577347.0          360856        9.139885e+08           NaN     578954  ,
 Empty DataFrame
 Columns: []
 Index: [])

In [25]:
def fetch_market_history_daily(coingecko_id, days=365):
    data = cg_get(
        f"/coins/{coingecko_id}/market_chart",
        params={"vs_currency":"usd", "days":days, "interval":"daily"},
        base_sleep=1.5,
        max_retries=5
    )

    prices = pd.DataFrame(data["prices"], columns=["ts_ms", "price_usd"])
    mcaps  = pd.DataFrame(data["market_caps"], columns=["ts_ms", "market_cap_usd"])
    vols   = pd.DataFrame(data["total_volumes"], columns=["ts_ms", "volume_usd"])

    df = prices.merge(mcaps, on="ts_ms").merge(vols, on="ts_ms")
    df["date"] = pd.to_datetime(df["ts_ms"], unit="ms", utc=True).dt.date
    df["coingecko_id"] = coingecko_id
    df = df.drop(columns=["ts_ms"])

    # one row per token per date
    df = df.groupby(["coingecko_id", "date"], as_index=False).mean(numeric_only=True)
    return df

history_frames = []
history_errors = []

for _, row in tm.iterrows():
    cid = str(row["coingecko_id"]).strip()
    try:
        history_frames.append(fetch_market_history_daily(cid, days=365))
    except Exception as e:
        history_errors.append({"coingecko_id": cid, "error": str(e)})

market_history_daily = pd.concat(history_frames, ignore_index=True)

market_history_daily.head(), len(history_errors)


Rate limited (429). Waiting 5s then retrying... (1/5)
Rate limited (429). Waiting 10s then retrying... (2/5)
Rate limited (429). Waiting 20s then retrying... (3/5)
Rate limited (429). Waiting 5s then retrying... (1/5)
Rate limited (429). Waiting 10s then retrying... (2/5)
Rate limited (429). Waiting 20s then retrying... (3/5)
Rate limited (429). Waiting 5s then retrying... (1/5)
Rate limited (429). Waiting 10s then retrying... (2/5)
Rate limited (429). Waiting 5s then retrying... (1/5)
Rate limited (429). Waiting 10s then retrying... (2/5)
Rate limited (429). Waiting 5s then retrying... (1/5)
Rate limited (429). Waiting 10s then retrying... (2/5)
Rate limited (429). Waiting 20s then retrying... (3/5)


(  coingecko_id        date  price_usd  market_cap_usd    volume_usd
 0      uniswap  2025-01-20  13.275081    7.997656e+09  6.670297e+08
 1      uniswap  2025-01-21  13.469417    8.100521e+09  7.125942e+08
 2      uniswap  2025-01-22  13.487985    8.099307e+09  4.728167e+08
 3      uniswap  2025-01-23  12.866830    7.729917e+09  3.142100e+08
 4      uniswap  2025-01-24  12.797972    7.688804e+09  3.640611e+08,
 0)

In [26]:
import os

# 1) Add recovered snapshot rows into the main snapshot df
recovered_df = pd.DataFrame(recovered)

market_snapshot_full = pd.concat([market_snapshot, recovered_df], ignore_index=True)

# 2) Remove duplicates (if any) keeping the latest pull per token
market_snapshot_full = market_snapshot_full.sort_values("pulled_at_utc").drop_duplicates("coingecko_id", keep="last")

# 3) Join metadata to both datasets
meta_cols = ["coingecko_id","token_name","symbol","tier","category","chain"]

market_snapshot_full = market_snapshot_full.merge(tm[meta_cols], on="coingecko_id", how="left")
market_history_daily = market_history_daily.merge(tm[meta_cols], on="coingecko_id", how="left")

# 4) Create folders + save
os.makedirs("data/processed", exist_ok=True)

market_snapshot_full.to_csv("data/processed/market_snapshot.csv", index=False)
market_history_daily.to_csv("data/processed/market_history_daily.csv", index=False)

print("Saved:")
print("data/processed/market_snapshot.csv")
print("data/processed/market_history_daily.csv")

market_snapshot_full.shape, market_history_daily.shape


Saved:
data/processed/market_snapshot.csv
data/processed/market_history_daily.csv


((20, 23), (7300, 10))

In [27]:
# should be 20 tokens
market_snapshot_full["coingecko_id"].nunique()


20

In [28]:
# should show 20 tokens and ~365 days each (some may have fewer)
market_history_daily.groupby("coingecko_id")["date"].count().describe()



count     20.0
mean     365.0
std        0.0
min      365.0
25%      365.0
50%      365.0
75%      365.0
max      365.0
Name: date, dtype: float64